In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

os.environ["GOOGLE_API_KEY"] = "REMOVED"
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash",temperature=0)

In [3]:
from crewai import Crew, Agent, Task

In [4]:
general_information = "Patient Name: Sneh Age: 20 Symptoms: - High fever (103 degrees Fahrenheit) - Cold - Cough Pain Assessment: None provided Medication: Paracetamol, effective for a few hours Symptom Duration: 2 days Additional Information: None"

In [5]:
vision_information = """
Skin Color and Condition:

Pale Skin: The patient's skin appears slightly pale, which could indicate anemia, shock, or lack of oxygen.
Yellowish Skin (Jaundice): No signs of yellowing are observed in the skin.
Blue Skin or Lips (Cyanosis): No blue discoloration is visible on the skin or lips.
Redness: There is significant redness on the cheeks and nose, which may suggest a condition like rosacea.
Rashes or Hives: No rashes or hives are visible on the face.
Pimples, Pustules, or Pox: No pimples, pustules, or pox are observed.
Eyes:

Dark Circles: There are dark circles under the eyes, suggesting sleep deprivation, stress, or chronic fatigue.
Yellowing (Scleral Icterus): No yellowing is observed in the sclera.
Red or Bloodshot Eyes: The eyes appear slightly red or bloodshot, which could indicate an infection, allergy, or irritation.
Drooping Eyelids (Ptosis): The eyelids appear slightly droopy, which may be due to fatigue or neurological issues.
Mouth and Lips:

Blue Lips (Cyanosis): The lips appear slightly blue, indicating poor oxygenation.
Swollen Lips: There is no swelling observed on the lips.
Cracked or Dry Lips: The lips appear dry, which may indicate dehydration or vitamin deficiencies.
Neck region:

Swollen Glands: No swollen glands are visible in the neck region.
Visible Thyroid Enlargement (Goiter): The thyroid gland is not visibly enlarged.
"""

In [18]:
symptom_assessment_agent = Agent(
    role="Symptom Assessment Agent",
    goal="To assess the patient's symptoms and categorize them as severe, moderate, or mild.",
    backstory="An analytical agent focused on identifying and categorizing symptoms based on severity.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

symptom_assessment_task = Task(
    description=(
        "1. Assess the patient's general information\n: {general_information} \n and visual information: \n{visual_information}."
        "2. Categorize the symptoms as severe, moderate, or mild."
        "3. Provide the corresponding score based on the provided rules: {symptom_rules}."
        "4. If the person has more than one symptoms, provide the score based on the most severe symptom."
    ),
    expected_output="Category and score based on the patient's symptoms.",
    agent=symptom_assessment_agent,
)

# Define the second agent: External Factors Scoring
external_factors_agent = Agent(
    role="External Factors Agent",
    goal="To evaluate additional factors such as age, symptom duration, and medication effectiveness.",
    backstory="A diligent agent designed to assess external factors that impact patient urgency.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

external_factors_task = Task(
    description=(
        "1. Evaluate the patient's general information: {general_information} ."
        "2. Assign additional points based on factors like age, symptom duration, and medication effectiveness according to the rules: {external_factors_rules}.\n"
        "3. Please make  you calculate the score for symptom duration correctly"
    ),
    expected_output="Additional score based on external factors.",
    agent=external_factors_agent,
)

# Define the third agent: Final Urgency Ranking
final_urgency_agent = Agent(
    role="Final Urgency Ranking Agent",
    goal="To combine the scores from symptom assessment and external factors to determine the patient's urgency level.",
    backstory="A precise agent that combines different scores to provide a final urgency ranking.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

final_urgency_task = Task(
    description=(
        "1. Combine the scores from symptom assessment: of symptom score agent and external factors: of external factor agent.\n"
        "2. Calculate the final urgency score based on the combined information and rules {final_ranking_rules}\n."
        "3. Make sure that you are combining the scores correctly.\n"
        "4. Provide a ranked score from 1 to 10 based on the urgency of the patient's condition.\n"
    ),
    expected_output="Final urgency score from 1 to 10.",
    context = [external_factors_task,symptom_assessment_task],
    agent=final_urgency_agent,
    
)

# Rules for each agent
symptom_rules = """
Severe but Not Immediately Life-Threatening Conditions:
Chest Pain (non-heart related): 3 points
High Fever  (e.g., 103°F or higher for more than 2 days): 3 points
Severe Abdominal Pain (potential appendicitis): 3 points
Uncontrolled Bleeding (e.g., deep cuts, heavy nosebleed): 4 points
Severe Dehydration (due to vomiting/diarrhea): 3 points

Moderate Conditions:
Moderate Fever (e.g., 102°F or lower): 2 points
Persistent Vomiting or Diarrhea: 2 points
Severe Headache (potential migraine): 2 points
Stomach Ache from Eating Stale Food: 2 points

Mild Conditions:
Swelling or Pain in Joints: 1 point
Mild Cold or Flu Symptoms: 1 point
Mild Allergic Reactions (mild rash, itching): 1 point
Minor Cuts and Scrapes: 1 point
Routine Check-ups and Follow-ups: 1 point
"""

external_factors_rules = """
Patient Age and Overall Health:
Elderly (age greater than 50 ): +1 points

Symptom Duration and Progression:
Symptoms for more than 3 days: +1 points

Effectiveness of Medication:
Medication not having any effect or having effect for very short time: +2 points
"""

final_ranking_rules = """
Ranking Calculation:
Initialize the score:
Start with 0 points.

Combine scores:
Add points from symptom assessment and external factors .
If the external factor score is 0, consider the symptom score only.

Determine the urgency level:
"""


In [19]:
crew = Crew(agents=[symptom_assessment_agent,external_factors_agent,final_urgency_agent],tasks=[symptom_assessment_task,external_factors_task,final_urgency_task],verbose=2)

2024-06-28 22:35:37,431 - 16668 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [20]:
inputs = {"general_information":general_information,"visual_information":vision_information,"symptom_rules":symptom_rules,"external_factors_rules":external_factors_rules,"final_ranking_rules":final_ranking_rules}
result = crew.kickoff(inputs)

 [2024-06-28 22:35:38][DEBUG]: == Working Agent: Symptom Assessment Agent
 [2024-06-28 22:35:38][INFO]: == Starting Task: 1. Assess the patient's general information
: Patient Name: Sneh Age: 20 Symptoms: - High fever (103 degrees Fahrenheit) - Cold - Cough Pain Assessment: None provided Medication: Paracetamol, effective for a few hours Symptom Duration: 2 days Additional Information: None 
 and visual information: 

Skin Color and Condition:

Pale Skin: The patient's skin appears slightly pale, which could indicate anemia, shock, or lack of oxygen.
Yellowish Skin (Jaundice): No signs of yellowing are observed in the skin.
Blue Skin or Lips (Cyanosis): No blue discoloration is visible on the skin or lips.
Redness: There is significant redness on the cheeks and nose, which may suggest a condition like rosacea.
Rashes or Hives: No rashes or hives are visible on the face.
Pimples, Pustules, or Pox: No pimples, pustules, or pox are observed.
Eyes:

Dark Circles: There are dark circles und